In [ ]:
library(remotes)
library(dplyr)
#install_github("chr1swallace/coloc",build_vignettes=TRUE)
library("coloc")

In [ ]:
snp_info<-read.csv("/path/of/annotation/snp_info_uniq.txt",sep = ',')
names(snp_info)=c("snpid","snp","chr","position","MAF",'anno','R','F')
head(snp_info)

In [ ]:
run_coloc_for_gene <- function(gene_name, df,significance_threshold = 0.75,gwas_number,percent_case,eqtl_number) {
  test <- df[df$gene == gene_name,]
  print (gene_name)
  my.res <- coloc.abf(
    dataset1 = list(snp = test$snpid,  beta = test$b, varbeta = test$var ,type = "cc",N= gwas_number,s = percent_case),
    dataset2 = list(snp = test$snpid, beta = test$beta, varbeta = test$beta_se2, MAF = test$maf, N = eqtl_number, type = "quant"),
    MAF = test$maf
  )
    my.res$summary$gene <- gene_name


  if (my.res$summary$PP.H4.abf >= significance_threshold) {
    #print(my.res$summary$PP.H4.abf)
    my.res$results$gene <- gene_name
    my.res$results$PP.H4.abf <-my.res$summary$PP.H4.abf 
    return(my.res$results)}
  else {
    return(NULL) 
  }

}

In [ ]:
gwas_type = "CD"
gwas <- read.csv(paste("/path/of/gwas/",gwas_type,"/",gwas_type,"_har_n.tsv",sep = ""),sep = '\t')
names(gwas) <- c('snpid','A1','A2','freq','b','se','p','case','conctol','n')

gwas$var <- (gwas$se)^2
gwas <- gwas[!duplicated(gwas[,c("snpid")]), ]
gwas <- gwas[gwas$b != 0, ]
gwas <- gwas[gwas$var != 0, ]
N = gwas$n[1]
s = gwas$case[1] / gwas$n[1]

In [ ]:
celltype="CD4-T"
chrom="6"
eqtl <- read.csv(paste("/path/of/tensorqtl/cis/",celltype,"_chr",chrom,".csv",sep = ""),row.names = 1)
eqtl$beta_se2 <- (eqtl$beta_se)^2
eqtl_df <- left_join(eqtl, snp_info, by="snpid")
eqtl_df <- eqtl_df[!duplicated(eqtl_df[,c("snpid","hervid")]), ]
df <- left_join(eqtl_df, gwas, by="snpid")
df <- na.omit(df)

In [ ]:
gene="MER65-int_dup45-chr6"
eqtl_Number = 981
res <- run_coloc_for_gene(gene, df ,gwas_number=N,percent_case=s,eqtl_number = eqtl_Number)

In [ ]:
print(head(res[order(res$SNP.PP.H4, decreasing = TRUE),]))